<a href="https://colab.research.google.com/github/alslmini/Machine-Learning-Programming/blob/main/20251104%EC%8B%A4%EC%8A%B5/CNN_Smartphone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# --- 데이터 로드 함수들 (변경 없음) ---

def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        # 파일 경로가 올바른지 확인을 위해 prefix + name을 출력해볼 수도 있습니다.
        # print(prefix + name)
        data = load_file(prefix + name)
        loaded.append(data)
    loaded = np.dstack(loaded)
    return loaded

def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'

    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']

    X = load_group(filenames, filepath)
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# --- load_dataset 함수 (경로 수정) ---

def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix)
    # load all test
    testX, testy = load_dataset_group('test', prefix)

    trainy = trainy - 1
    testy = testy - 1

    trainy_one_hot = to_categorical(trainy)
    testy_one_hot = to_categorical(testy)

    print(f"trainX: {trainX.shape}, trainy: {trainy.shape}, trainy_one_hot: {trainy_one_hot.shape}")
    print(f"testX: {testX.shape}, testy: {testy.shape}, testy_one_hot: {testy_one_hot.shape}")
    return trainX, trainy, trainy_one_hot, testX, testy, testy_one_hot

DATASET_ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/Machine-Learning/20251104/UCI HAR Dataset/'

print("데이터셋 로드 중...")
trainX, trainy, trainy_one_hot, testX, testy, testy_one_hot = load_dataset(DATASET_ROOT_PATH)
print("데이터셋 로드 완료.")

# train 데이터를 학습/검증 셋으로 분리
X_train,X_val,y_train_one_hot,y_val_one_hot,y_train,y_val=train_test_split(trainX, trainy_one_hot, trainy,test_size=0.2,random_state=100)

print("\n--- 학습/검증 셋 분리 결과 ---")
print(f"X_train.shape: {X_train.shape}")
print(f"X_val.shape: {X_val.shape}")
print(f"y_train_one_hot.shape: {y_train_one_hot.shape}")
print(f"y_val_one_hot.shape: {y_val_one_hot.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_val.shape: {y_val.shape}")

데이터셋 로드 중...


/tmp/ipython-input-3778035332.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
/tmp/ipython-input-3778035332.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
/tmp/ipython-input-3778035332.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
/tmp/ipython-input-3778035332.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = pd.read_csv(filepath, header=None, delim_whitespace=Tr

trainX: (7352, 128, 9), trainy: (7352, 1), trainy_one_hot: (7352, 6)
testX: (2947, 128, 9), testy: (2947, 1), testy_one_hot: (2947, 6)
데이터셋 로드 완료.

--- 학습/검증 셋 분리 결과 ---
X_train.shape: (5881, 128, 9)
X_val.shape: (1471, 128, 9)
y_train_one_hot.shape: (5881, 6)
y_val_one_hot.shape: (1471, 6)
y_train.shape: (5881, 1)
y_val.shape: (1471, 1)
